In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Definir la EDO
def model(t, z, mu):
    return mu * z - z**2

# Solución analítica para mu > 0
def analytical_solution(t, mu, z0):
    denominator = z0 + (mu - z0) * np.exp(-mu * t)
    return (mu * z0) / denominator

# Implementación de los métodos numéricos
def euler_method(f, t_span, z0, n, mu):
    t0, tf = t_span
    t = np.linspace(t0, tf, n)
    h = t[1] - t[0]
    z = np.zeros(n)
    z[0] = z0
    
    for i in range(n-1):
        z[i+1] = z[i] + h * f(t[i], z[i], mu)
    return t, z

def improved_euler_method(f, t_span, z0, n, mu):
    t0, tf = t_span
    t = np.linspace(t0, tf, n)
    h = t[1] - t[0]
    z = np.zeros(n)
    z[0] = z0
    
    for i in range(n-1):
        k1 = f(t[i], z[i], mu)
        k2 = f(t[i] + h, z[i] + h * k1, mu)
        z[i+1] = z[i] + (h/2) * (k1 + k2)
    return t, z

def runge_kutta_4(f, t_span, z0, n, mu):
    t0, tf = t_span
    t = np.linspace(t0, tf, n)
    h = t[1] - t[0]
    z = np.zeros(n)
    z[0] = z0
    
    for i in range(n-1):
        k1 = f(t[i], z[i], mu)
        k2 = f(t[i] + h/2, z[i] + h*k1/2, mu)
        k3 = f(t[i] + h/2, z[i] + h*k2/2, mu)
        k4 = f(t[i] + h, z[i] + h*k3, mu)
        z[i+1] = z[i] + (h/6) * (k1 + 2*k2 + 2*k3 + k4)
    return t, z

# Función para resolver numéricamente con diferentes métodos
def solve_numerically(method, mu, z0, t_span, n_points=100):
    t_eval = np.linspace(t_span[0], t_span[1], n_points)
    
    if method == 'Euler':
        t, z = euler_method(model, t_span, z0, n_points, mu)
    elif method == 'Euler mejorado':
        t, z = improved_euler_method(model, t_span, z0, n_points, mu)
    elif method == 'Runge-Kutta 4':
        t, z = runge_kutta_4(model, t_span, z0, n_points, mu)
    
    return z

# Parámetros interactivos
@interact(
    mu=FloatSlider(value=1.0, min=-2.0, max=2.0, step=0.1, description='μ'),
    z0=FloatSlider(value=0.5, min=0.0, max=3.0, step=0.1, description='z0'),
    method=['Euler', 'Euler mejorado', 'Runge-Kutta 4']
)
def plot_interactive(mu, z0, method):
    t_span = (0, 10)
    t_eval = np.linspace(0, 10, 100)
    
    # Solución numérica
    z_numerical = solve_numerically(method, mu, z0, t_span)
    
    # Solución analítica (solo para mu > 0)
    if mu > 0:
        z_analytical = analytical_solution(t_eval, mu, z0)
    else:
        z_analytical = None

    # Gráfico
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(t_eval, z_numerical, 'b-', label=f'Numérica ({method})')
    if z_analytical is not None:
        plt.plot(t_eval, z_analytical, 'r--', label='Analítica')
    plt.xlabel('Tiempo')
    plt.ylabel('Población z(t)')
    plt.title(f'Solución para μ={mu}, z0={z0}')
    plt.legend()
    plt.grid(True)
    
    # Diagrama de bifurcación
    plt.subplot(1, 2, 2)
    mu_values = np.linspace(-2, 2, 100)
    z_eq1 = np.zeros_like(mu_values)  # z=0
    z_eq2 = mu_values  # z=mu
    plt.plot(mu_values, z_eq1, 'b-', label='z=0', linewidth=2)
    plt.plot(mu_values[mu_values < 0], z_eq2[mu_values < 0], 'r--', label='z=μ (inestable)', linewidth=2)
    plt.plot(mu_values[mu_values >= 0], z_eq2[mu_values >= 0], 'g-', label='z=μ (estable)', linewidth=2)
    plt.xlabel('μ')
    plt.ylabel('z')
    plt.title('Diagrama de Bifurcación Transcrítica')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='μ', max=2.0, min=-2.0), FloatSlider(value=0.5, descr…